The necessary libraries for this script are not included in the requirement file. You will need to install additionally plotly and nbformat. You can use the following commands:

```bash
pip install plotly
pip install nbformat
```

**Warning**: On Windows 10, the export as svg sometimes runs endlessly without producing an output. If that the case, reinstall kaleido with `pip install --upgrade "kaleido==0.1.*"`.

In [ ]:
import os
import pathlib
import sys

import pandas as pd
import plotly.graph_objects as go
import numpy as np

sys.path.insert(1, '..')
import util_las as las

In [ ]:
WORKING_DIR = '../..'
os.chdir(WORKING_DIR)

In [ ]:
os.getcwd()

## Sankey Diagram visualisation:

In [ ]:

NEW_PATH = 'data/LiDAR_data/2022_Neuchatel/lidar2022_classified/2546500_1212000.laz'
CLASSES_CORRESPONDENCE_PATH = 'data/class_equivalences.csv'
PLOTS_FOLDER_PATH = 'outputs/plots'
save_to_html = True
save_to_image = True

newer_pc= las.las_to_df_xyzclass(NEW_PATH)

In [ ]:
points_per_class = newer_pc.groupby('classification')['classification'].count().to_frame('nb_points').reset_index()
points_per_class.rename(columns={'classification':'id'},inplace=True)
points_per_class.head()

In [ ]:
class_equivalences = pd.read_csv(CLASSES_CORRESPONDENCE_PATH, sep=';')

In [ ]:
class_equivalences['index_new']=class_equivalences.index
class_equivalences.head(4)

In [ ]:
prev_class_only = pd.DataFrame({'matched_id':[1,2,3,6,7,9,17,-1], 'common_class_name':['Unclassified (1)','Ground (2)','Vegetation (3)','Building (6)','Noise (7)','Water (9)','Bridge (17)','Removed (-1)']})
prev_class_only.index+=len(class_equivalences) # So that we have an index which works with sankey
prev_class_only['index_prev']=prev_class_only.index
prev_class_only.head(3)

In [ ]:
class_equivalences=class_equivalences.merge(prev_class_only, how='left', on='matched_id')
class_equivalences.head()

In [ ]:
sankey_df = class_equivalences.merge(points_per_class, how='inner',on='id')
sankey_df.head()

In [ ]:
all_diagram_elements = pd.concat([class_equivalences[['id','class_name']],prev_class_only.rename(columns={'matched_id':'id','common_class_name':'class_name'})],ignore_index=True)
all_diagram_elements.tail(9)

In [ ]:
fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = all_diagram_elements.class_name,
      #color = "blue"
    ),
    link = dict(
      source = sankey_df.index_new,
      target = sankey_df.index_prev,
      value = sankey_df.nb_points
  ))])

fig.update_layout(
    autosize=False,
    width=800,
    #height=800,
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [ ]:
os.path.basename(NEW_PATH).split('.')[0]

In [ ]:
if save_to_html:
    pathlib.Path(PLOTS_FOLDER_PATH).mkdir(parents=True, exist_ok=True)
    plot_name = os.path.basename(NEW_PATH).split('.')[0] + '_sankey_flow_of_classes.html'
    fig.write_html(os.path.join(PLOTS_FOLDER_PATH, plot_name), full_html=False, include_plotlyjs='cdn')

In [ ]:
if save_to_image:
    pathlib.Path(PLOTS_FOLDER_PATH).mkdir(parents=True, exist_ok=True)
    print('ping')
    plot_name = os.path.basename(NEW_PATH).split('.')[0] + '_sankey_flow_of_classes.svg'
    print('ping')
    fig.write_image(os.path.join(PLOTS_FOLDER_PATH,plot_name))
    print('ping')